## 二、数据及研究方法

在本部分，首先介绍了数据的来源，然后详细说明了 B-score 的构建方法以及检验 B-score 有效性的方法。

### （一）数据获取

使用 CSMAR 数据库，时间范围选取 2014-2023 年的全市场股票财务数据。

需要的原始数据包括：资产总计、负债总计、长期负债合计、优先股、所有者权益合计、净利润、营业利润、利润总额、市盈率、月收盘价

其中，资产总计、负债总计、长期负债合计、优先股、所有者权益合计、净利润、营业利润、利润总额的单位均为百万（元）

In [33]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [34]:
def fetch_data(rel_path):
    files = os.listdir(rel_path)
    csv_name = [x for x in files if x.split(".")[1] == "csv"][0]
    txt_name = [x for x in files if x.split(".")[1] == "txt"][0]
    txt_file = open(os.path.join(rel_path, txt_name), encoding="utf-8")
    csv_data = pd.read_csv(os.path.join(rel_path, csv_name), dtype={0: str})
    content = txt_file.readlines()
    name_list = []
    for line in content:
        code, name = line.split(" [")
        name = name.split("]")[0]
        name_list.append(name)

    # rename columns
    csv_data.columns = name_list

    # clean the data
    csv_data = csv_data.fillna(0)
    if "交易月份" in csv_data.columns:
        csv_data["交易月份"] = pd.to_datetime(csv_data["交易月份"])
    else:
        csv_data["统计截止日期"] = pd.to_datetime(csv_data["统计截止日期"])

    # use type A report
    if "报表类型" in name_list:
        csv_data = csv_data[csv_data["报表类型"] == "A"]

    return csv_data

raw_data = {}
mapping = {"bal": "资产负债表", "inc": "利润表", "pe": "相对价值指标", "ret": "月个股回报率", "day": "公布日期"}
for item in list(mapping.keys()):
    path = f'../data/{mapping[item]}'
    raw_data[item] = fetch_data(path)

考虑到年/季报晚于统计截止日期，为了消除前瞻偏差（look- ahead bias），需要对数据做特殊处理

In [35]:
def get_calendar(start_day="2014-01-31", end_day="2023-12-31"):
    cal = pd.date_range(start=start_day, end=end_day, freq="M")
    cal = cal.strftime("%Y-%m")
    return cal.tolist()

def get_symbols(raw_data):
    symbol_lists = []
    for item in list(raw_data.keys()):
        tmp_df = raw_data[item]
        index_name = "证券代码" if "证券代码" in tmp_df.columns else "股票代码"
        symbol_lists.append(tmp_df[index_name].tolist())
    tmp_set = set()
    for i in range(len(symbol_lists)):
        if i == 0:
            tmp_set = set(symbol_lists[i])
        else:
            tmp_set = tmp_set & set(symbol_lists[i])  # intersaction

    return sorted(list(tmp_set))

def init_df(start_day="2014-01-31", end_day="2023-12-31"):
    return pd.DataFrame(0, index=get_calendar(start_day=start_day, end_day=end_day), 
                        columns=get_symbols(raw_data))

def store_csv(df, name, rel_dir=None):
    output_dir = f"../output/{rel_dir}/" if rel_dir else "../output/"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    df.to_csv(output_dir + name + ".csv")

In [36]:
def process_data(raw_data, key, variables=None, rel_dir=None, start_day="2013-12-31", end_day="2023-12-31"):
    clean_data = {}
    tmp_df = init_df(start_day=start_day, end_day=end_day)
    if variables is None:
        variables = raw_data[key].columns[4:]
    abs_dir = f"../output/{rel_dir}/" if rel_dir else "../output/"
    if not os.path.exists(abs_dir):
        os.mkdir(abs_dir)
    for var in tqdm(variables, desc="variables"):
        if f"{var}.csv" in os.listdir(abs_dir):  # skip existed
            clean_data[var] = pd.read_csv(abs_dir +var + ".csv", index_col=0)
            continue
        index_name = "证券代码" if "证券代码" in raw_data[key].columns else "股票代码"  # syntax diff in pe
        tmp_tb = pd.pivot(raw_data[key], index=index_name, columns="统计截止日期", values=var)
        report_df = raw_data["day"].groupby(["证券代码", "统计截止日期"])["报告公布日期"].first()
        report_df = pd.to_datetime(report_df, errors='coerce')
        report_df = report_df.dropna()
        report_df = report_df.dt.strftime("%Y-%m")

        for (symbol, day), month in report_df.items():  # match by unique symbol and day
            if (month in tmp_df.index) & (symbol in tmp_df.columns):
                tmp_df.loc[month, symbol] = tmp_tb.loc[symbol, day]
        
        tmp_df = tmp_df.replace(0, np.nan).ffill()
        if np.mean(tmp_df) > 10e6:
            tmp_df = tmp_df / 10e6
        store_csv(tmp_df, name=var, rel_dir=rel_dir)
        
        clean_data[var] = tmp_df

    return clean_data

In [37]:
pe_data = process_data(raw_data, "pe", ["市盈率（PE）TTM"])
bal_data = process_data(raw_data, "bal")
inc_data = process_data(raw_data, "inc")

variables: 100%|██████████| 3/3 [00:23<00:00,  7.89s/it]


In [18]:
def reframe_close_df(raw_data):
    """
    Change the form of monthly close price
    """
    if os.path.exists("../output/月收盘价.csv"):
        return
    ret_df = raw_data["ret"]
    ret_df["交易月份"] = pd.to_datetime(ret_df["交易月份"]).dt.strftime("%Y-%m")
    ret_tb = ret_df.groupby(["证券代码", "交易月份"])["月收盘价"].first()
    ret_data = init_df()
    months = [x[1] for x in ret_tb.index]
    symbols = [x[0] for x in ret_tb.index]
    for (symbol, month), ret in ret_tb.items():
        if (month in months) & (symbol in symbols):
            ret_data.loc[month, symbol] = ret

    store_csv(ret_data, "月收盘价")

In [38]:
def get_describe():
    desc_df = pd.DataFrame()
    output_items = os.listdir("../output/")
    csv_items = [x for x in output_items if "csv" in x]
    all_variables = [x.split(".")[0] for x in csv_items]
    for var in tqdm(all_variables):
        tmp_df = pd.read_csv(f"../output/{var}.csv", index_col=0)
        tmp_des = tmp_df.melt().replace(0, np.nan).dropna().describe().T
        tmp_des.index = [var]
        desc_df = pd.concat([desc_df, tmp_des])
    desc_df = desc_df.drop(["count", "min", "max"], axis=1)
    return desc_df

**表 1 Panel A**

In [39]:
# table 1 panel A
desc_df = get_describe()
store_csv(desc_df, "描述性统计", rel_dir="panels")
desc_df

100%|██████████| 10/10 [00:04<00:00,  2.21it/s]


,mean,std,25%,50%,75%
营业利润,77.970743,772.162637,1.042319e+00,6.316671e+00,2.320382e+01
其中：优先股,4.389566,170.623449,1.731122e-19,3.753629e-19,9.959482e-19
月收盘价,19.892204,36.000529,6.720000e+00,1.188000e+01,2.178000e+01
市盈率（PE）TTM,238.032403,6052.217837,2.200479e+01,4.085255e+01,8.765868e+01
长期负债合计,177.694499,2739.931450,1.974635e-05,5.343835e-05,6.168373e-04
负债合计,1866.393708,49589.487700,2.351365e-12,7.582105e-12,2.023344e+01
净利润,20.966489,389.033993,1.526073e-14,1.203300e-13,5.524540e-12
所有者权益合计,346.698348,4817.855587,2.333059e-26,7.499211e-26,6.499873e+01
利润总额,26.013645,481.666577,1.546893e-07,1.209554e-06,6.055000e-05
资产总计,6957.008519,93502.144521,1.732145e+02,3.753905e+02,9.962668e+02


In [40]:
desc_df.to_excel("../output/panels/描述性统计.xlsx")

### （二）B-score 的构建

**1. Safety 维度**

$$
Safety=\frac{1}{leverage}=\frac{1}{0.38\times MLEV+0.35\times DTOA+0.27\times BLEV}
$$

其中

$$
MLEV=\frac{log(普通股市场价值)+优先股+长期负债}{log(普通股市场价值)}
$$

$$
BLEV=\frac{log(普通股账面价值)+优先股+长期负债}{log(普通股账面价值)}
$$

$$
DTOA=\frac{log(总负债)}{log(总资产)}
$$

其中普通股市场价值用上一个交易日的股票收盘价计算，其余数据用上一财年的年报计算。普通股账面价值=所有者权益总计-优先股

**2. Cheapness 维度**

$$
PEG=\frac{市盈率(TTM)}{利润增长率\times 100}
$$

其中

$$
利润增长率=\frac{利润总额_{n+1}}{利润总额_n}-1
$$

**3. Quality 维度**

$$
Quality=\frac{净利润}{营业利润}
$$

In [57]:
data = {}
all_variables = os.listdir("../output/")
for var in all_variables:
    if ".csv" in var:
        tmp_df = pd.read_csv("../output/"+var, index_col=0)
        data[var.split(".")[0]] = tmp_df.fillna(0)

In [59]:
def sigmoid(x):
    """
    x is a dataframe
    """
    return 1 / (1 + np.exp(-x))

def condom(x):
    return x.fillna(0).replace([np.inf, -np.inf], 0)

def mlev():
    return 1 + (data["其中：优先股"] + data["长期负债合计"]) / np.log(data["月收盘价"])

def blev():
    return 1 + (data["其中：优先股"] + data["长期负债合计"]) / np.log(data["所有者权益合计"])

def dtoa():
    return np.log(data["负债合计"]) / np.log(data["资产总计"])

def Safety():
    safety = (0.38 * mlev() + 0.35 * dtoa() + 0.27 * blev()) * 10e-10
    return sigmoid(condom(safety))

def earning_growth():
    return data["利润总额"].pct_change()

def Cheapness():
    cheapness = data["市盈率（PE）TTM"] / earning_growth().fillna(np.inf)
    return sigmoid(condom(cheapness))

def Quality():
    quality = data["净利润"] / data["营业利润"]
    return sigmoid(condom(quality))

def B_score():
    return sigmoid(Safety() + Cheapness() + Quality())

**相关性矩阵**

首先用每两个指标的在每个横截面下的数据计算相关性系数，接着将所有横截面下的计算结果取均值，表示这两个指标的相关性，构建相关性矩阵如下

In [60]:
func_list = [B_score(), Safety(), Cheapness(), Quality()]

def get_corr(m1, m2):
    cal = get_calendar()
    coef_list = []
    for day in cal:
        coef = np.corrcoef(m1.loc[day, :].values, m2.loc[day, :].values)
        coef_list.append(coef[0][1])
    return np.mean([x for x in coef_list if not np.isnan(x)])

def get_corr_matrix(func_list):
    names = ["B_score", "Safety", "Cheapness", "Quality"]
    num = len(func_list)
    corr_matrix = [[0 for _ in range(num)] for _ in range(num)]
    for i in range(num):
        for j in range(i+1, num):
            corr_matrix[i][j] = get_corr(func_list[i], func_list[j])
    corr_matrix = np.array(corr_matrix)
    corr_matrix[range(num), range(num)] = 1
    return pd.DataFrame(corr_matrix, index=names, columns=names)

**表 1 Panel B**

In [61]:
corr_mat = get_corr_matrix(func_list)
store_csv(corr_mat, "相关性矩阵", rel_dir="panels")
corr_mat

,B_score,Safety,Cheapness,Quality
B_score,1.0,0.21515,0.714420,0.364088
Safety,0.0,1.00000,0.111054,-0.025442
Cheapness,0.0,0.00000,1.000000,-0.054042
Quality,0.0,0.00000,0.000000,1.000000


## 三、实证结果分析

### （一）、单变量分组检验

首先计算各分组未经调整的收益

注意由于我们用 n-1 期月末的因子值排序构建组合，在 n 期月末观测到其收益率，这里存在一期的错位。因此用 shift 函数将所有因子值向下一期平移，对齐数据

In [62]:
def get_group_returns(day, score):
    """
    Specified the score used for sorting, monthly return of each portfolio
    day: str, format "YYYY-mm" 
    score: dataframe
    return:
    - avg_w: np.array, lens=5
    - size_w_df: np.array, lens=5
    - num_sample: int
    """
    score = score.shift(1).iloc[1:]  # lag-1
    df = score.loc[day] # B_score, Safety, Cheapness, Quality
    month_ret = condom(data["月收盘价"].pct_change().iloc[1:]).loc[day]
    merge_df = pd.concat([df, month_ret], axis=1)
    merge_df.columns = ["score", "return"]
    merge_df["size"] = data["月收盘价"].loc[day]
    merge_df ["return*size"] = merge_df ["return"] * merge_df["size"]
    merge_df = merge_df[merge_df["size"] != 0]  # public only
    merge_df = merge_df.sort_values("score", ascending=False)
    merge_df["group"] = pd.qcut(range(len(merge_df)), q=5, labels=[f"P{i+1}" for i in range(5)])
    avg_w = merge_df.groupby("group")["return"].mean().values
    size_w = (merge_df.groupby("group")["return*size"].mean() / merge_df.groupby("group")["size"].sum()).values
    num_sample = len(merge_df)
    
    return avg_w, size_w, num_sample

def get_portfolio_return(score):
    """
    Iterate across time
    score: dataframe
    return: pd.DataFrame(index=days, columns=['P1',...,'P5','P5-P1'])
    """
    avg_w_list = []
    size_w_list = []
    num_list = []
    for day in get_calendar():
        avg_w, size_w, num_sample = get_group_returns(day, score)
        avg_w_list.append(avg_w)
        size_w_list.append(size_w)
        num_list.append(num_sample)

    avg_w_df = pd.DataFrame(avg_w_list, columns=[f"P{i+1}" for i in range(5)], index=[get_calendar()])
    size_w_df = pd.DataFrame(size_w_list, columns=[f"P{i+1}" for i in range(5)], index=[get_calendar()])
    avg_w_df["P5-P1"] = avg_w_df["P5"] - avg_w_df["P1"]
    size_w_df["P5-P1"] = size_w_df["P5"] - size_w_df["P1"]

    return avg_w_df, size_w_df

def get_unadjusted_return(avg_w_df, size_w_df):
    """
    Specified the score used for sorting, calculate unadj cumulative return and t-stat
    return: 
    - ret: list, len=6
    - t_stat: float
    """
    avg_w_ret = ((1 + avg_w_df).cumprod() - 1).tail(1).values[0]
    size_w_ret = ((1 + size_w_df).cumprod() - 1).tail(1).values[0]
    avg_t_stat, p_value = stats.ttest_1samp(avg_w_df["P5-P1"], 0)
    size_t_stat, p_value = stats.ttest_1samp(size_w_df["P5-P1"], 0)

    return avg_w_ret, size_w_ret, avg_t_stat, size_t_stat

In [67]:
indicator_mapping = {
    "Safety": Safety(),
    "Cheapness": Cheapness(),
    "Quality": Quality(),
    "B_score": B_score()
}

def single_fac_test_unadj(indicator_mapping):
    """
    Iterate across all scores used for sorting
    """
    portfolio_name = [f"P{i+1}" for i in range(5)] + ["P5-P1"]
    avg_output = pd.DataFrame(0., columns=portfolio_name, index=list(indicator_mapping.keys()))
    size_output = pd.DataFrame(0., columns=portfolio_name, index=list(indicator_mapping.keys()))

    for ind in tqdm(list(indicator_mapping.keys())):
        avg_w_df, size_w_df = get_portfolio_return(indicator_mapping[ind])
        avg_w_ret, size_w_ret, avg_t, size_t = get_unadjusted_return(avg_w_df, size_w_df)
        avg_output.loc[ind, portfolio_name] = avg_w_ret
        size_output.loc[ind, portfolio_name] = size_w_ret
        avg_output.loc[ind, "t值"] = avg_t
        size_output.loc[ind, "t值"] = size_t
    
    unadjusted_output = pd.concat([avg_output, size_output], keys=["等额加权", "市值加权"]).T
    store_csv(unadjusted_output, "未经风险调整的收益率", rel_dir="panels")

    return unadjusted_output

In [68]:
single_fac_test_unadj(indicator_mapping)

100%|██████████| 4/4 [00:07<00:00,  1.76s/it]


等额加权                                    市值加权                      \
         Safety Cheapness   Quality   B_score    Safety Cheapness   Quality   
P1     1.347738  0.607319  0.242480  1.425341  0.006795  0.004673  0.003376   
P2     0.312052  0.434393  0.126532  0.801547  0.003261  0.004452  0.003170   
P3     0.576033  1.082233  0.586672  0.682153  0.003902  0.005325  0.003548   
P4     0.436596  0.898076  0.658766  0.392213  0.003363  0.004083  0.003506   
P5     0.882229  0.556092  2.915906  0.340830  0.005667  0.004402  0.010007   
P5-P1 -0.192943 -0.005228  2.090271 -0.447487 -0.001121 -0.000271  0.006610   
t值     0.047590  0.132629  3.917884 -0.979392 -0.507434 -0.313425  3.962761   

                 
        B_score  
P1     0.006963  
P2     0.004249  
P3     0.005114  
P4     0.003382  
P5     0.004110  
P5-P1 -0.002834  
t值    -2.162860

接着，计算经过 Fama-French 三因子调整的收益率

MarkettypeID 选择 P9714：沪深A股和创业板和科创板

每个月按照上个月末的 B-score 指标将股票分成 5 组。如果按照 B-score 构造的 5 档组合月平均收益率呈现单调趋势且多空组合有显著正回报，那么可以认为 B-score 指标对横截面上股票收益率的变化具有解释能力。

Fama-French三因子模型用于调整投资组合的收益率，以消除市场、规模和账面市值比等因素的影响。Fama-French三因子模型的公式如下：

$$
R_{it} - R_{ft} = \alpha_i + \beta_{iM} \cdot (R_{Mt} - R_{ft}) + \beta_{iSMB} \cdot SMB_t + \beta_{iHML} \cdot HML_t + \epsilon_{it}
$$

$$
\text{调整后收益率} = R_{it} - \left[ \beta_{iM} (R_{Mt} - R_{ft}) + \beta_{iSMB} \cdot SMB_t + \beta_{iHML} \cdot HML_t \right]
$$

In [44]:
def get_rf():
    """
    Load risk-free rate from CSMAR
    """
    tmp_df = pd.read_csv("../data/无风险利率/BND_Exchange.csv", index_col=0)
    tmp_df.index = pd.to_datetime(tmp_df.index)
    monthly_rf = tmp_df.resample("M").mean()
    monthly_rf.index = monthly_rf.index.strftime("%Y-%m")
    monthly_rf = monthly_rf.loc[get_calendar()]
    
    return monthly_rf.values.flatten()[1:]  # skip 2013-12

def get_adjusted_return(fmf, w_df):
    """
    fmf: three factor df of type=P9714
    w_df: pd.DataFrame(index=days, columns=["P1",..."P5","P5-P1"])
    return:
    - adj_ret_list: cumulative adjusted return of portfolios P1,...,P5,P5-P1
    - t_stat: t statistic of portfolio P5-P1
    """
    adj_ret_list = []
    fmf = fmf.iloc[1:]  # omit 2013-12
    fmf = fmf[["RiskPremium1", "SMB1", "HML1"]].values
    x = sm.add_constant(fmf)
    t_stat = 0
    r_f = get_rf()
    for col in w_df.columns.tolist():
        r_it = w_df[col].iloc[1:].values
        y = r_it - r_f  # minus risk-free
        # create model
        model = sm.OLS(y, x)
        result = model.fit()
        # adjusted return
        adj_return = r_it - np.sum(result.params[1:] * fmf, axis=1)
        cum_return = (1 + adj_return).cumprod()
        adj_ret_list.append(cum_return[-1])
        if col == "P5-P1":
            t_stat, p_value = stats.ttest_1samp(r_it, 0)
            
    return adj_ret_list, t_stat

def single_fac_test_adj(indicator_mapping, fmf):
    """
    Iterate across all scores used for sorting
    """
    portfolio_name = [f"P{i+1}" for i in range(5)] + ["P5-P1"]
    avg_output = pd.DataFrame(0., columns=portfolio_name, index=list(indicator_mapping.keys()))
    size_output = pd.DataFrame(0., columns=portfolio_name, index=list(indicator_mapping.keys()))
    for ind in list(indicator_mapping.keys()):
        avg_w_df, size_w_df = get_portfolio_return(indicator_mapping[ind])
        avg_w_ret, avg_t = get_adjusted_return(fmf, avg_w_df)
        size_w_ret, size_t = get_adjusted_return(fmf, size_w_df)
        avg_output.loc[ind, portfolio_name] = avg_w_ret
        size_output.loc[ind, portfolio_name] = size_w_ret
        avg_output.loc[ind, "t值"] = avg_t
        size_output.loc[ind, "t值"] = size_t

    adjusted_output = pd.concat([avg_output, size_output], keys=["等额加权", "市值加权"]).T
    store_csv(adjusted_output, "Fama-French三因子调整收益率", rel_dir="panels")

    return adjusted_output

In [45]:
# import fama-french data
fmf = pd.read_csv("../data/三因子模型指标/STK_MKT_THRFACMONTH.csv")
fmf = fmf[fmf["MarkettypeID"] == "P9714"].reset_index(drop=True)
single_fac_test_adj(indicator_mapping, fmf)

等额加权                                    市值加权                      \
         Safety Cheapness   Quality   B_score    Safety Cheapness   Quality   
P1     0.802090  0.922758  0.646268  0.737043  1.238751  1.241399  1.238814   
P2     0.770319  0.791396  0.631391  0.889725  1.238643  1.240710  1.238678   
P3     0.733821  0.767390  0.830085  0.808384  1.238547  1.238975  1.239112   
P4     0.719558  1.157840  0.752752  0.875449  1.238740  1.240364  1.238720   
P5     1.527528  0.780479  1.983522  1.066297  1.243672  1.239431  1.246053   
P5-P1  2.256453  0.994311  3.701257  1.711697  1.241887  1.235010  1.244202   
t值     1.120006 -0.574097  3.603918  0.798359  2.538315 -1.109296  3.710579   

                 
        B_score  
P1     1.238627  
P2     1.240510  
P3     1.239553  
P4     1.240050  
P5     1.241726  
P5-P1  1.240068  
t值     2.543563

### （二）Fama-Macbeth 回归

为了进一步检验 B-score 对股票收益率的预测能力，本部分采用了 Fama-Macbeth 回归，以便同时控制住其他影响横截面股票收益率的公司特征。回归模型中被解释变量是个股月收益率，解释变量包括 B-score、Safety、Cheapness 以及 Quality；控制变量包括对数市值（SIZE）、账面市值比（BM）。回归模型如下：

$$
\begin{aligned}
r_{i,t}=&a+b_1 \text{B\_score}_{i,t-1}+b_2 Safety_{i,t-1}+b_3 Cheapness_{i, t-1}\\
&+b_4 Quality_{i,t-1}+b_5 SIZE_{i,t-1}+b_6 BM_{i,t-1}+\epsilon
\end{aligned}
$$

In [334]:
raw_data["control"] = fetch_data("../data/控制变量/")
control_data = process_data(raw_data, key="control", variables=["市值A", "账面市值比A"], rel_dir="control",
                            start_day="2013-12-31", end_day="2023-12-31")
control_data["市值A"] = np.log(control_data["市值A"])

variables: 100%|██████████| 2/2 [00:16<00:00,  8.27s/it]


数据框格式为 pd.DataFrame(index=days, columns=symbols)，每个变量对应的存储地址为

- $r_{i,t}$: r_it
- B_score: B_score()
- Safety: Safety()
- Cheapness: Cheapness()
- Quality: Quality()
- Size: control_data["市值A"]
- BM: control_data["账面市值比A"]

另外需注意时间对齐问题。将收益率向上一期平移，对齐到 t-1 期

In [648]:
fm_mapping = {
    "B_score": condom(B_score()),
    "Safety": condom(Safety()),
    "Cheapness": condom(Cheapness()),
    "Quality": condom(Quality()),
    "Size": condom(control_data["市值A"]),
    "BM": condom(control_data["账面市值比A"])
}

In [675]:
def fm_stage_one(symbol, variables, valid_days, r_it):
    # x = np.column_stack([fm_mapping[key].loc[valid_days, symbol].values for key in variables])
    x = np.transpose(np.array([fm_mapping[key].loc[valid_days, symbol].values for key in variables]))
    x = sm.add_constant(x)
    y = r_it[valid_days].values
    model = sm.OLS(y, x)
    results = model.fit()

    return results.params[1:]

def run_fm_one(variables, modelid=None):
    params_list = []
    ret_data = pd.read_csv("../output/月收盘价.csv", index_col=0)
    ret_data = condom(ret_data.pct_change().shift(-1).dropna(axis=0, how="all"))
    symbols = []
    abs_dir = f"../output/fm/Model {modelid}.csv"
    if os.path.exists(abs_dir):
        betas = pd.read_csv(abs_dir, index_col=0)
        symbols = betas.columns.tolist()
        return betas.T, symbols
    
    for symbol in tqdm(get_symbols(raw_data)):
        r_it = ret_data.loc[:, symbol]
        valid_days = r_it[r_it != 0].index.tolist()
        if valid_days:
            params = fm_stage_one(symbol, variables, valid_days, r_it)
            if len(params) == len(variables):
                params_list.append(params)
                symbols.append(symbol)

    betas = pd.DataFrame(params_list, index=symbols, columns=variables).T
    store_csv(betas, f"Model {modelid}", rel_dir="fm")

    return betas.T, symbols

def fm_stage_two(betas, symbols):
    ret_data = pd.read_csv("../output/月收盘价.csv", index_col=0)
    ret_data = ret_data.pct_change().shift(-1).dropna(axis=0, how="all")
    fil_ret_data = condom(ret_data.loc[:, symbols])
    days = fil_ret_data.index.tolist()
    lambda_list = []
    for day in days:
        y_series = fil_ret_data.loc[day]
        y_series = y_series[y_series != 0]
        valid_symbols = y_series.index.tolist()
        y = y_series.values
        x = betas.loc[valid_symbols].values
        x = sm.add_constant(x)
        
        model = sm.OLS(y, x)
        result = model.fit()
        lambda_list.append(result.params[1:])
    
    mean = np.mean(lambda_list, axis=0)
    t_stat, _ = stats.ttest_1samp(lambda_list, 0)

    return mean, t_stat

def fm_reg(variables, jobid):
    """
    Combine functions:
    - fm_stage_one
    - run_fm_one
    - fm_stage_two
    """
    betas, symbols = run_fm_one(variables, modelid=jobid)
    mean, t_stat = fm_stage_two(betas, symbols)

    return mean, t_stat

In [676]:
fm_reg(["B_score"], jobid=1)
fm_reg(["Safety"], jobid=2)
fm_reg(["Cheapness"], jobid=3)
fm_reg(["Quality"], jobid=4)
fm_reg(["B_score", "Size", "BM"], jobid=5)
fm_reg(["Safety", "Cheapness", "Quality", "Size", "BM"], jobid=6)

100%|██████████| 5493/5493 [00:08<00:00, 637.93it/s]


(array([-5.13218784e-03, -3.96225604e-02, -6.52209799e-03,  2.96415197e-01,
         7.93283615e+07]),
 array([-3.44358031, -1.67605564, -2.86528293,  0.85058113,  1.27574884]))

In [671]:
def fm_results(id=0):
    assert (id == 0) | (id == 1), "id should be 0 or 1"
    fm_coef = pd.DataFrame(0., index=list(fm_mapping.keys()), columns=[f"Model {i+1}" for i in range(6)])
    fm_coef.loc["B_score", "Model 1"] = fm_reg(["B_score"], jobid=1)[id]
    fm_coef.loc["Safety", "Model 2"] = fm_reg(["Safety"], jobid=2)[id]
    fm_coef.loc["Cheapness", "Model 3"] = fm_reg(["Cheapness"], jobid=3)[id]
    fm_coef.loc["Quality", "Model 4"] = fm_reg(["Quality"], jobid=4)[id]
    fm_coef.loc[["B_score", "Size", "BM"], "Model 5"] = fm_reg(["B_score", "Size", "BM"], jobid=5)[id]
    fm_coef.loc[["Safety", "Cheapness", "Quality", "Size", "BM"], "Model 6"] = fm_reg(["Safety", "Cheapness", "Quality", "Size", "BM"], jobid=6)[id]
    return fm_coef

系数

In [672]:
fm_results(id=0)

100%|██████████| 5493/5493 [00:16<00:00, 329.48it/s]


,Model 1,Model 2,Model 3,Model 4,Model 5,Model 6
B_score,-0.013035,0.000000,0.000000,0.000000,-1.024888e-02,0.000000e+00
Safety,0.000000,-0.010303,0.000000,0.000000,0.000000e+00,-5.132188e-03
Cheapness,0.000000,0.000000,-0.045644,0.000000,0.000000e+00,-3.962256e-02
Quality,0.000000,0.000000,0.000000,-0.013239,0.000000e+00,-6.522098e-03
Size,0.000000,0.000000,0.000000,0.000000,2.075644e-02,2.964152e-01
BM,0.000000,0.000000,0.000000,0.000000,2.874911e+07,7.932836e+07


t值

In [673]:
fm_results(id=1)

,Model 1,Model 2,Model 3,Model 4,Model 5,Model 6
B_score,-3.572402,0.000000,0.000000,0.00000,-3.072997,0.000000
Safety,0.000000,-4.335901,0.000000,0.00000,0.000000,-3.443580
Cheapness,0.000000,0.000000,-1.900103,0.00000,0.000000,-1.676056
Quality,0.000000,0.000000,0.000000,-3.16685,0.000000,-2.865283
Size,0.000000,0.000000,0.000000,0.00000,0.048990,0.850581
BM,0.000000,0.000000,0.000000,0.00000,0.444976,1.275749
